In [2]:
import numpy as np
import pandas as pd


In [27]:
df = pd.read_csv('filtered_features.csv').drop('target', axis=1)
model_forecasts = pd.read_csv('results_LSTM_2.csv')
model_forecasts.head()

,Date,0,1,2,3,4,5,6,7,8,9
0,2013-02-15,1518.4331,1518.5924,1518.5815,1518.5034,1518.4021,1518.2996,1518.2053,1518.1227,1518.0520,1517.9927
1,2013-02-19,1519.5847,1519.9994,1519.9358,1519.7196,1519.4485,1519.1799,1518.9354,1518.7225,1518.5420,1518.3915
2,2013-02-20,1519.3943,1519.0707,1518.8453,1518.6445,1518.4786,1518.3405,1518.2264,1518.1328,1518.0562,1517.9938
3,2013-02-21,1517.7191,1516.9133,1516.6909,1516.6812,1516.7720,1516.8951,1517.0222,1517.1394,1517.2427,1517.3306
4,2013-02-22,1516.7438,1516.4489,1516.4363,1516.5468,1516.6986,1516.8552,1517.0001,1517.1276,1517.2362,1517.3274


In [28]:
df.head()

,Unnamed: 0,Date,^GSPC_Open,^GSPC_Volume,^GSPC_SQZ_20_2.0_20_1.5,^GSPC_SQZ_ON,^GSPC_ABER_ATR_5_15,^GSPC_ADOSC_3_10,^GSPC_ADX_14,^GSPC_DMP_14,...,DX-Y.NYB_VHF_28,DX-Y.NYB level 5,DX-Y.NYB level 10,DX-Y.NYB level 20,DX-Y.NYB level 25,DX-Y.NYB level 30,DX-Y.NYB level 50,^GSPC_High,^GSPC_Low,^GSPC_Close
0,0,1993-01-29,438.670013,247200000,7.049998,False,2.902007,6.886895e+07,18.006061,28.075804,...,0.244481,1.0,1.0,0.0,0.0,1.0,0.0,438.929993,436.910004,438.779999
1,1,1993-02-01,438.779999,238570000,7.881663,False,2.957872,1.703827e+08,18.460351,34.209112,...,0.237145,0.0,0.0,0.0,0.0,0.0,0.0,442.519989,438.779999,442.519989
2,2,1993-02-02,442.519989,271560000,7.234996,False,2.901346,2.566182e+08,18.970118,33.292546,...,0.260181,0.0,0.0,0.0,0.0,0.0,0.0,442.869995,440.760010,442.549988
3,3,1993-02-03,442.559998,345410000,7.144994,False,3.027924,3.715875e+08,20.417289,40.078885,...,0.278434,1.0,0.0,0.0,0.0,0.0,0.0,447.350006,442.559998,447.200012
4,4,1993-02-04,447.200012,351140000,8.356664,False,3.003394,4.713655e+08,22.193214,43.509068,...,0.316940,1.0,0.0,0.0,0.0,0.0,0.0,449.859985,447.200012,449.559998


In [14]:
model_forecasts['trend_forecasts'] = model_forecasts.apply(lambda x: 1 if x[1] < x[model_forecasts.shape[1] - 1] else -1 , axis=1)
model_forecasts['trend_forecasts'].value_counts()

 1    1668
-1     816
Name: trend_forecasts, dtype: int64

In [15]:
model_forecasts = model_forecasts.drop([f'{i}' for i in range(5, 10)], axis=1)
model_forecasts.head()


,Date,0,1,2,3,4,trend_forecasts
0,2013-02-15,1518.4331,1518.5924,1518.5815,1518.5034,1518.4021,-1
1,2013-02-19,1519.5847,1519.9994,1519.9358,1519.7196,1519.4485,-1
2,2013-02-20,1519.3943,1519.0707,1518.8453,1518.6445,1518.4786,-1
3,2013-02-21,1517.7191,1516.9133,1516.6909,1516.6812,1516.7720,-1
4,2013-02-22,1516.7438,1516.4489,1516.4363,1516.5468,1516.6986,1


In [4]:
tick = ['^GSPC', '^IXIC', '^VIX', 'DX-Y.NYB']
model_types = ['v2_BiLSTM']
for model_type in model_types:
    for num in [1]:
        df = pd.read_csv('filtered_features.csv').drop(['Unnamed: 0', 'target'], axis=1)
        model_forecasts = pd.read_csv(f'new_results_{model_type}_{num}.csv')
        model_forecasts.head()
        model_forecasts['trend_forecast'] = model_forecasts.apply(lambda x: 1 if np.sum(x[1:]) > 0 else -1 , axis=1)
        model_forecasts['trend_forecast_next_day'] = model_forecasts.apply(lambda x: 1 if x[1] > 0 else -1 , axis=1)
        print(model_forecasts['trend_forecast'].value_counts())
        print(model_forecasts['trend_forecast_next_day'].value_counts())
        
        model_forecasts = model_forecasts.drop([f'{i}' for i in range(5, 10)], axis=1)
        model_forecasts.head()
        df = pd.merge(df, model_forecasts, 'inner', 'Date')
        df.to_csv(f'filtered_features_with_forecasts_{model_type}_{num}.csv')
        k = 0
        for i in tick:

            columns = [f'{i}_Open',f'{i}_High',f'{i}_Low',f'{i}_Close',f'{i}_Adj Close',
            f'{i}_Volume',f'{i}_EMA_5',f'{i}_EMA_10',f'{i}_EMA_20',f'{i}_EMA_50',f'{i}_EMA_100',f'{i}_EMA_200',f'{i}_SMA_200',
            f'{i}_MACD_12_26_9' ,f'{i} level 50', f'{i} % from high', 'Date'
            ]
            if k == 0:
                d_tick = pd.read_csv(f'{i}_ta_my_features.csv').drop(['target'], axis= 1)
                d_tick=d_tick.dropna(axis=1).reset_index(drop=True)

                df2 = pd.merge(d_tick, model_forecasts, 'inner', 'Date')
                df2.to_csv(f'{i}_ta_my_features_with_forecasts_{model_type}_{num}.csv')
                df2 = d_tick
                d_tick[columns].to_csv(f'{i}_some_my_features.csv')
                df3 = d_tick[columns]

                df4 = pd.merge(d_tick[columns], model_forecasts, 'inner', 'Date')
                df4.to_csv(f'{i}_some_my_features_with_forecasts_{model_type}_{num}.csv')
            else:
                d_tick = pd.read_csv(f'{i}_ta_my_features.csv')
                df3 = pd.merge(df3, d_tick[columns], 'inner', 'Date')
                df2 = pd.merge(df2, d_tick, 'inner', 'Date')
            k += 1

        df3 = df3.dropna(axis=1).reset_index(drop=True)
        df3.to_csv('all_some_my_features.csv')
        df3 = pd.merge(df3, model_forecasts, 'inner', 'Date')
        df3.to_csv(f'all_some_my_features_with_forecasts_{model_type}_{num}.csv')
        
        df2 = df2.dropna(axis=1).reset_index(drop=True)
        df2.to_csv('all_ta_features.csv')
        df2 = pd.merge(df2, model_forecasts, 'inner', 'Date')
        df2.to_csv(f'all_ta_features_with_forecasts_{model_type}_{num}.csv')


'''
Итого следующие наборы данных:
filtered_features.csv
filtered_features_with_forecasts_LSTM_i.csv

^GSPC_ta_my_features.csv
^GSPC_ta_my_features_with_forecasts_LSTM_i.csv

^GSPC_some_my_features.csv
^GSPC_some_my_features_with_forecasts_LSTM_i.csv

all_some_my_features.csv
all_some_my_features_with_forecasts_LSTM_i.csv

all_ta_features.csv
all_ta_features_with_forecasts_LSTM_i.csv
'''

 1    1968
-1     516
Name: trend_forecast, dtype: int64
 1    1791
-1     693
Name: trend_forecast_next_day, dtype: int64


'\nИтого следующие наборы данных:\nfiltered_features.csv\nfiltered_features_with_forecasts_LSTM_i.csv\n\n^GSPC_ta_my_features.csv\n^GSPC_ta_my_features_with_forecasts_LSTM_i.csv\n\n^GSPC_some_my_features.csv\n^GSPC_some_my_features_with_forecasts_LSTM_i.csv\n\nall_some_my_features.csv\nall_some_my_features_with_forecasts_LSTM_i.csv\n\nall_ta_features.csv\nall_ta_features_with_forecasts_LSTM_i.csv\n'

In [5]:
model_forecasts.head()

,Date,0,1,2,3,4,trend_forecast,trend_forecast_next_day
0,2013-02-15,0.001568,0.001460,0.001416,0.001397,0.001388,1,1
1,2013-02-19,0.001095,0.001263,0.001330,0.001358,0.001370,1,1
2,2013-02-20,0.002127,0.001690,0.001516,0.001442,0.001409,1,1
3,2013-02-21,0.002227,0.001740,0.001540,0.001454,0.001414,1,1
4,2013-02-22,0.001326,0.001368,0.001379,0.001380,0.001379,1,1


In [32]:
df = pd.read_csv('^GSPC_ta_my_features.csv')
df.head()

,Unnamed: 0,Date,^GSPC_Open,^GSPC_High,^GSPC_Low,^GSPC_Close,^GSPC_Adj Close,^GSPC_Volume,^GSPC_EMA_5,^GSPC_EMA_10,...,^GSPC Last Close 2,^GSPC Last Close 3,^GSPC Last Close 4,^GSPC Last Close 5,^GSPC Last Close 6,^GSPC Last Close 7,^GSPC Last Close 8,^GSPC Last Close 9,^GSPC Last Close 10,target
0,0,1993-01-29,438.670013,438.929993,436.910004,438.779999,438.779999,247200000,438.395204,437.498333,...,438.109985,439.950012,440.010010,436.109985,435.489990,433.369995,435.130005,436.839996,437.149994,0
1,1,1993-02-01,438.779999,442.519989,438.779999,442.519989,442.519989,238570000,439.770132,438.411362,...,438.660004,438.109985,439.950012,440.010010,436.109985,435.489990,433.369995,435.130005,436.839996,0
2,2,1993-02-02,442.519989,442.869995,440.760010,442.549988,442.549988,271560000,440.696751,439.163839,...,438.779999,438.660004,438.109985,439.950012,440.010010,436.109985,435.489990,433.369995,435.130005,0
3,3,1993-02-03,442.559998,447.350006,442.559998,447.200012,447.200012,345410000,442.864504,440.624961,...,442.519989,438.779999,438.660004,438.109985,439.950012,440.010010,436.109985,435.489990,433.369995,0
4,4,1993-02-04,447.200012,449.859985,447.200012,449.559998,449.559998,351140000,445.096336,442.249513,...,442.549988,442.519989,438.779999,438.660004,438.109985,439.950012,440.010010,436.109985,435.489990,1


In [28]:
df2=df.dropna(axis=1).reset_index(drop=True)

In [30]:
df2.to_csv('^GSPC_ta_my_features.csv')